# SSY340 Project - Animal Detection Networks for Univeseum
Group 8: Zhuqi Xiao, Yizhe Gu

**Note**：To run this notebook, please follow the setps listed in [YOLOv5 Enviornment](#env) to have its enviornment. 

## Dataset annotation

The very first step of our project is to collect and annotate our dataset, which is very critical step. 

For this project, we went to the museum in person and manually collected 764 images of real scenes. The most popular formats for annotating an object detection datasets are Pascal VOC, Micosoft COCO, and YOLO format. Since we intend to use transfer learning, YOLO and Faster R-CNN require different formats of annotations, so two different annotation formats have been prepared, VOC and YOLO formats. 

Rearding the labelling tool, we used an open source tool for our job. 

Link to [labelImg](https://github.com/tzutalin/labelImg.git) on github.

Steps to install:
```shell
$ git clone https://github.com/tzutalin/labelImg.git
$ cd labelImg
$ pip3 install pyqt5 lxml
$ make qt5py3
$ python3 labelImg.py
$ # python3 labelImg.py [IMAGE_PATH] [PRE-DEFINED CLASS FILE]
```

To initiate a label, type `w`, and draw the intended label. Then, type ctrl (or command) `S` to save the label. Type `d` to go to the next image (and `a` to go back an image).

### YOLO format of labeling
Export your labels to YOLO format, with one `*.txt` file per image (if no objects in image, no `*.txt` file is required). The `*.txt` file specifications are:

+ One row per object
+ Each row is `class x_center y_center width height` format.
+ Box coordinates must be in **normalized xywh** format (from 0 - 1). If your boxes are in pixels, divide `x_center` and `width` by image width, and `y_center` and `height` by image height.
+ Class numbers are zero-indexed (start from 0).

### Create data.yaml
This *yaml* file is needed when you want to train a network or want to test your trained network. It provides your dataset info to the training or testing script. 

The file format is shown as below:
```yaml
# dataset.yaml file format
# File hierarchy
#   parent
#   ├── yolov5
#   ├── datasets
#       ├── train
#           ├── images
#           └── labels
#       ├── val
#           ├── images
#           └── labels
#       ├── test
#           ├── images
#           └── labels
#       └── data.yaml


# Train/val/test sets as 
# 1) dir: path/to/imgs, 
# 2) file: path/to/imgs.txt, or 
# 3) list: [path/to/imgs1, path/to/imgs2, ..]

path: ../datasets/data  # dataset root dir
train: images/train  # train images (relative to 'path')
val: images/val  # val images (relative to 'path')
test: images/test

# Classes
nc: 19  # number of classes
names: ['Arapaima', 'Blue Fish', 'Clownfish', 'Common Dragonet', 'Emerald Toucanet', 'Goldbelly Damsel', 'Indian Surgeonfish', 'Lemon Damsel', 'Orange Revabborre', 'Picked Dogfish', 'Pied Tamarin', 'Red Ibis', 'Silver Moony', 'Star Fish', 'Stingray', 'Sunbittern', 'Thornback Ray', 'Toco Toucan', 'Yellowfin Surgeonfish'] # class labels
```

### Labeling Best Practices ([From web](https://blog.roboflow.com/labelimg/#:~:text=Labeling%20Best%20Practices))
When labeling images, there are a few tips to bear in mind. Note that these tips are generally true, but can alter based on the context of your problem.

+ Label around the entirety of an object. It is best to include a little bit of non-object buffer than it is to exclude a portion of the object with a rectangular label. So, aim to have boxes that tightly mirror the objects you want to label, but do not cut off part of the objects. Your model will understand edges far better this way.
+ For occluded objects, label them entirely. If an object is out of view due to another object being in front of it, label the object out of view as if you could see its entirety. Your model will begin to understand the true bounds of objects this way.
+ For objects partially out of frame, generally label them. This tip especially depends on your problem, but in general, even a partial object is still an object to be labeled.


## YOLOv5 Enviornment<div id="env"></div>
[YOLOv5 Link](https://github.com/ultralytics/yolov5)

YOLOv5 is an open-source project that consists of a familyof  object  detection  models. In the official repository of Yolov5 on Github, there are a total of 4 versions of the object detection networks, namely `Yolov5s`, `Yolov5m`, `Yolov5l`, and `Yolov5x`, which  represents different sizes of YOLOv5. Besides, training and testing codes are provided. 

Followings are for preparing an enviornment for running the code and assuminig you are using a conda enviornment. 

+ Create a new virtual conda environment
    ```sh
    $ conda create --name yolov5 # create a new virtual env
    $ conda activate yolov5 # activate the new created env
    $ conda install pip # install pip for later use
    ```

+ Get pre-trained YOLOv5 model
    ```sh
    $ git clone https://github.com/ultralytics/yolov5  # clone repo
    $ cd yolov5
    $ pip install -r requirements.txt  # install required packages
    ```

Now everything should have been prepared and good to go on. 

## Transfer Learning

Since we only have a small dataset, in order to obtain a good model in time, the transfer learning method is adopted. We separately train on pre-trained YOLOv5 model and Faster RCNN model. (In this notebook, only YOLO part is included)

Since the model will be potentially deployed on mobile phones, smaller models are preferred. Therefore, the smallest YOLOv5 architecture, **YOLOv5s**, is then selected to be trained on. 

In [1]:
%cd yolov5/

/home/zhuqi/deep-machine-learning/project/SSY340-Project/yolov5


In [2]:
!python val.py --data ../data/data.yaml --weights runs/train/yolov5s_results10/weights/best.pt --img 640

val: data=../data/data.yaml, weights=['runs/train/yolov5s_results10/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=val, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False
YOLOv5 🚀 v6.0-30-gfee83c1 torch 1.10.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
/home/zhuqi/conda/envs/yolov5/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 232 layers, 7295064 parameters, 0 gradients, 16.9 GFLOPs
val: Scanning '/home/zhuqi/deep-machine-learning/project/SSY340-Project/data/val
               Class     Images     Labels          P          R     mAP@.5 mAP@
 

In [3]:
!python val.py --data ../data/data.yaml --weights runs/train/yolov5s_results10/weights/best.pt --img 640 --task test

val: data=../data/data.yaml, weights=['runs/train/yolov5s_results10/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, task=test, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False
YOLOv5 🚀 v6.0-30-gfee83c1 torch 1.10.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
/home/zhuqi/conda/envs/yolov5/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 232 layers, 7295064 parameters, 0 gradients, 16.9 GFLOPs
test: Scanning '/home/zhuqi/deep-machine-learning/project/SSY340-Project/data/te
test: New cache created: /home/zhuqi/deep-machine-learning/project/SSY340-Project

In [4]:
!python detect.py --source ../data/IMG_0201.mp4 --weights runs/train/yolov5s_results10/weights/best.pt

detect: weights=['runs/train/yolov5s_results10/weights/best.pt'], source=../data/IMG_0201.mp4, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.0-30-gfee83c1 torch 1.10.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
/home/zhuqi/conda/envs/yolov5/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 232 layers, 7295064 parameters, 0 gradients, 16.9 GFLOPs
video 1/1 (1/686) /home/zhuqi/deep-machine-learn

video 1/1 (36/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 2 Clownfishs, 2 Goldbelly Damsels, 1 Lemon Damsel, 4 Orange Revabborres, Done. (0.025s)
video 1/1 (37/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 2 Clownfishs, 2 Goldbelly Damsels, 1 Lemon Damsel, 4 Orange Revabborres, Done. (0.024s)
video 1/1 (38/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 2 Clownfishs, 2 Goldbelly Damsels, 1 Lemon Damsel, 4 Orange Revabborres, Done. (0.024s)
video 1/1 (39/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 2 Clownfishs, 2 Goldbelly Damsels, 1 Lemon Damsel, 4 Orange Revabborres, Done. (0.025s)
video 1/1 (40/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 2 Clownfishs, 2 Goldbelly Damsels, 1 Lemon Damsel, 4 Orange Rev

video 1/1 (76/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 3 Blue Fishs, 1 Clownfish, 1 Goldbelly Damsel, 1 Lemon Damsel, 7 Orange Revabborres, Done. (0.025s)
video 1/1 (77/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 3 Blue Fishs, 2 Clownfishs, 1 Goldbelly Damsel, 1 Lemon Damsel, 6 Orange Revabborres, Done. (0.025s)
video 1/1 (78/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 3 Blue Fishs, 2 Clownfishs, 1 Goldbelly Damsel, 1 Lemon Damsel, 6 Orange Revabborres, Done. (0.025s)
video 1/1 (79/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 3 Blue Fishs, 3 Clownfishs, 1 Goldbelly Damsel, 1 Lemon Damsel, 6 Orange Revabborres, Done. (0.024s)
video 1/1 (80/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 3 Blue Fishs, 2 Clownfishs, 1 Goldbelly Damsel, 1 Lemon Damsel, 7 Orange Revabborr

video 1/1 (115/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 3 Clownfishs, 2 Goldbelly Damsels, 1 Indian Surgeonfish, 1 Lemon Damsel, 7 Orange Revabborres, Done. (0.025s)
video 1/1 (116/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 3 Clownfishs, 3 Goldbelly Damsels, 1 Indian Surgeonfish, 1 Lemon Damsel, 7 Orange Revabborres, Done. (0.025s)
video 1/1 (117/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 3 Clownfishs, 2 Goldbelly Damsels, 1 Indian Surgeonfish, 1 Lemon Damsel, 9 Orange Revabborres, Done. (0.025s)
video 1/1 (118/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 2 Clownfishs, 2 Goldbelly Damsels, 1 Indian Surgeonfish, 1 Lemon Damsel, 9 Orange Revabborres, Done. (0.025s)
video 1/1 (119/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_02

video 1/1 (153/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 4 Blue Fishs, 1 Clownfish, 3 Goldbelly Damsels, 10 Orange Revabborres, Done. (0.025s)
video 1/1 (154/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 1 Clownfish, 3 Goldbelly Damsels, 10 Orange Revabborres, Done. (0.025s)
video 1/1 (155/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 2 Clownfishs, 4 Goldbelly Damsels, 1 Indian Surgeonfish, 9 Orange Revabborres, Done. (0.025s)
video 1/1 (156/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 2 Clownfishs, 4 Goldbelly Damsels, 1 Indian Surgeonfish, 9 Orange Revabborres, Done. (0.025s)
video 1/1 (157/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 2 Clownfishs, 5 Goldbelly Damsels, 1 Indian Surgeonfish, 9 Orange Revabborres,

...

video 1/1 (524/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 2 Clownfishs, 4 Goldbelly Damsels, 3 Lemon Damsels, 8 Orange Revabborres, Done. (0.025s)
video 1/1 (525/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 3 Blue Fishs, 2 Clownfishs, 4 Goldbelly Damsels, 4 Lemon Damsels, 8 Orange Revabborres, Done. (0.025s)
video 1/1 (526/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 4 Blue Fishs, 2 Clownfishs, 4 Goldbelly Damsels, 3 Lemon Damsels, 8 Orange Revabborres, Done. (0.025s)
video 1/1 (527/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 3 Blue Fishs, 2 Clownfishs, 3 Goldbelly Damsels, 1 Lemon Damsel, 8 Orange Revabborres, Done. (0.025s)
video 1/1 (528/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 3 Blue Fishs, 2 Clownfishs, 4 Goldbelly Damsels, 1 Lemon Damsel, 8 Or

video 1/1 (565/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 3 Blue Fishs, 1 Clownfish, 3 Goldbelly Damsels, 2 Lemon Damsels, 8 Orange Revabborres, Done. (0.024s)
video 1/1 (566/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 3 Blue Fishs, 1 Clownfish, 3 Goldbelly Damsels, 2 Lemon Damsels, 8 Orange Revabborres, Done. (0.025s)
video 1/1 (567/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 3 Blue Fishs, 1 Clownfish, 3 Goldbelly Damsels, 1 Lemon Damsel, 8 Orange Revabborres, Done. (0.025s)
video 1/1 (568/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 3 Blue Fishs, 1 Clownfish, 4 Goldbelly Damsels, 2 Lemon Damsels, 8 Orange Revabborres, Done. (0.025s)
video 1/1 (569/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 3 Blue Fishs, 1 Clownfish, 3 Goldbelly Damsels, 2 Lemon Damsels, 8 Orange

video 1/1 (606/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 1 Clownfish, 2 Goldbelly Damsels, 9 Orange Revabborres, Done. (0.025s)
video 1/1 (607/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 1 Clownfish, 2 Goldbelly Damsels, 9 Orange Revabborres, Done. (0.025s)
video 1/1 (608/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 1 Clownfish, 2 Goldbelly Damsels, 9 Orange Revabborres, Done. (0.025s)
video 1/1 (609/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 1 Clownfish, 3 Goldbelly Damsels, 9 Orange Revabborres, Done. (0.025s)
video 1/1 (610/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 1 Clownfish, 3 Goldbelly Damsels, 1 Lemon Damsel, 10 Orange Revabborres, Done. (0.025s)
video 1/1 (611/686) /home/zhuqi/deep-m

video 1/1 (647/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 1 Clownfish, 3 Goldbelly Damsels, 1 Indian Surgeonfish, 9 Orange Revabborres, Done. (0.025s)
video 1/1 (648/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 1 Clownfish, 3 Goldbelly Damsels, 1 Indian Surgeonfish, 9 Orange Revabborres, Done. (0.025s)
video 1/1 (649/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 1 Clownfish, 3 Goldbelly Damsels, 1 Indian Surgeonfish, 9 Orange Revabborres, Done. (0.025s)
video 1/1 (650/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 1 Clownfish, 2 Goldbelly Damsels, 1 Indian Surgeonfish, 8 Orange Revabborres, Done. (0.025s)
video 1/1 (651/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 1 Clownfish, 3 Goldbelly Damsels, 1 In

video 1/1 (686/686) /home/zhuqi/deep-machine-learning/project/SSY340-Project/data/IMG_0201.mp4: 384x640 2 Blue Fishs, 1 Clownfish, 2 Goldbelly Damsels, 1 Indian Surgeonfish, 6 Orange Revabborres, Done. (0.025s)
Speed: 0.5ms pre-process, 24.6ms inference, 1.6ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp
